In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from collections import Counter
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.utils.class_weight import compute_class_weight


2025-05-03 18:41:03.554028: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746297663.788151      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746297663.856853      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/mephi-hack2/cleared_one_class.csv')
# df = pd.read_csv('../data/processed/cleared_one_class.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4191 entries, 0 to 4190
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4191 non-null   object
 1   class   4191 non-null   object
dtypes: object(2)
memory usage: 65.6+ KB


In [3]:
df.head()

,text,class
0,⭐️ кнопка: ⭐️start⭐️(https://t.me/major/start...,соцсети
1,а продолжение где? правильно. в моем сообществ...,соцсети
2,тем временем моя авторская телега уверенно в т...,соцсети
3,"у меня есть двоюродная сестра, у нее есть сын ...",личная жизнь
4,тем временем моя авторская телега уверенно в т...,соцсети


In [4]:
# Инициализируем энкодер
encoder = LabelEncoder()

labels =encoder.fit_transform(df['class'])
df['labels'] = labels
df.head()

,text,class,labels
0,⭐️ кнопка: ⭐️start⭐️(https://t.me/major/start...,соцсети,3
1,а продолжение где? правильно. в моем сообществ...,соцсети,3
2,тем временем моя авторская телега уверенно в т...,соцсети,3
3,"у меня есть двоюродная сестра, у нее есть сын ...",личная жизнь,0
4,тем временем моя авторская телега уверенно в т...,соцсети,3


In [5]:
class_mapping = {label: idx for idx, label in enumerate(encoder.classes_)}
class_mapping

{'личная жизнь': 0,
 'политика': 1,
 'реклама': 2,
 'соцсети': 3,
 'спорт': 4,
 'юмор': 5}

In [6]:
df=df.drop(columns=['class'])
train_df, val_df = train_test_split(df, stratify=df['labels'], random_state=42)

In [7]:
# Находим количество примеров в каждом классе
class_counts = train_df['labels'].value_counts()
max_count = class_counts.max()

# Функция для оверсэмплинга каждого класса
resampled_dfs = []
for class_label in class_counts.index:
    class_df = train_df[train_df['labels'] == class_label]
    if len(class_df) < max_count:
        # Дублируем примеры, пока не достигнем max_count
        class_df = class_df.sample(max_count, replace=True, random_state=42)
    resampled_dfs.append(class_df)

# Объединяем обратно
train_df = pd.concat(resampled_dfs, axis=0).sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
texts = train_df['text'].to_numpy()
labels = train_df['labels'].to_numpy()
print(Counter(labels))

Counter({1: 979, 0: 979, 4: 979, 3: 979, 2: 979, 5: 979})


In [9]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train' : train_dataset,
    'val': val_dataset
})

In [10]:
# Токенизация
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Map:   0%|          | 0/5874 [00:00<?, ? examples/s]

Map:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets.set_format('torch')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5874
    })
    val: Dataset({
        features: ['text', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1048
    })
})

In [12]:
# 5. Инициализация модели (остается без изменений)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased',
    num_labels=6
).to(device)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Создание модели
model = BertForSequenceClassification.from_pretrained(
    "DeepPavlov/rubert-base-cased", 
    num_labels=len(class_mapping)
    )


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Для multi-class
    
    # Для multi-label (раскомментировать):
    # predictions = (predictions > 0).astype(int)  # Логиты уже через sigmoid
    
    return {
        'f1 macro': f1_score(labels, predictions, average='macro'),
        'f1 micro': f1_score(labels, predictions, average='micro'),
        'f1 weighted': f1_score(labels, predictions, average='weighted'),
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions, average='macro'),
        'recall': recall_score(labels, predictions, average='macro'),
        'Loss': hamming_loss(labels, predictions)
    }

training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    eval_strategy="steps",
    log_level='info',
    do_train=True,  # Ключевое изменение!
    do_eval=True,
    logging_steps=200,
    save_steps=500,
    eval_steps=200,
    fp16=True,  # Если GPU поддерживает
    report_to=["tensorboard"],
    metric_for_best_model='eval_f1 micro'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_metrics,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--DeepPavlov--rubert-base-cased/snapshots/4036cab694767a299f2b9e6492909664d9414229/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "p

In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 5,874
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Training with DataParallel so batch size has been adjusted to: 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3,680
  Number of trainable parameters = 177,858,054
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,F1 macro,F1 micro,F1 weighted,Accuracy,Precision,Recall
200,0.840100,0.278626,0.699041,0.721374,0.715083,0.721374,0.709564,0.708721
400,0.261400,0.269084,0.710895,0.730916,0.728045,0.730916,0.727292,0.703191
600,0.144300,0.269084,0.706626,0.730916,0.725945,0.730916,0.722974,0.698021
800,0.159800,0.260496,0.723936,0.739504,0.739508,0.739504,0.727794,0.729848
1000,0.141200,0.265267,0.700980,0.734733,0.728930,0.734733,0.699165,0.714949
1200,0.116700,0.239504,0.735320,0.760496,0.755472,0.760496,0.746994,0.731653
1400,0.081500,0.255725,0.729284,0.744275,0.743234,0.744275,0.728946,0.736252
1600,0.080000,0.252863,0.726695,0.747137,0.746698,0.747137,0.740501,0.718848
1800,0.097700,0.266221,0.721849,0.733779,0.734508,0.733779,0.727974,0.723507
2000,0.076500,0.268130,0.709653,0.731870,0.730432,0.731870,0.708572,0.717911


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1048
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1048
  Batch size = 16
/usr/local/lib/python3.11/dist-packages

TrainOutput(global_step=3680, training_loss=0.12869091665615207, metrics={'train_runtime': 6664.8279, 'train_samples_per_second': 17.627, 'train_steps_per_second': 0.552, 'total_flos': 3.091139690766336e+16, 'train_loss': 0.12869091665615207, 'epoch': 20.0})

In [17]:
model.save_pretrained("/kaggle/working/my_bert_one_aug_classifier")
tokenizer.save_pretrained("/kaggle/working/my_bert_one_aug_classifier")

Configuration saved in /kaggle/working/my_bert_one_aug_classifier/config.json
Model weights saved in /kaggle/working/my_bert_one_aug_classifier/model.safetensors
tokenizer config file saved in /kaggle/working/my_bert_one_aug_classifier/tokenizer_config.json
Special tokens file saved in /kaggle/working/my_bert_one_aug_classifier/special_tokens_map.json


('/kaggle/working/my_bert_one_aug_classifier/tokenizer_config.json',
 '/kaggle/working/my_bert_one_aug_classifier/special_tokens_map.json',
 '/kaggle/working/my_bert_one_aug_classifier/vocab.txt',
 '/kaggle/working/my_bert_one_aug_classifier/added_tokens.json')

In [18]:
print(trainer.evaluate()) 

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1048
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 4.122381687164307, 'eval_f1 macro': 0.7308203053209227, 'eval_f1 micro': 0.7452290076335878, 'eval_f1 weighted': 0.7449970772321786, 'eval_accuracy': 0.7452290076335878, 'eval_precision': 0.7336519775713232, 'eval_recall': 0.7317164762423718, 'eval_Loss': 0.2547709923664122, 'eval_runtime': 19.8025, 'eval_samples_per_second': 52.923, 'eval_steps_per_second': 3.333, 'epoch': 20.0}


In [19]:
new_texts = ["Анкара Месси забил гол в ворота франции", "ай Литвин красава ай чисто на кондциях залетел ай да лев", "Америка расширила список санкция против России", "купите макбук за 140к и получите наушники в подарок"]


In [20]:
def predict(text):
    # Токенизация текста
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    
    # Предсказание
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получение метки класса
    probs = torch.softmax(outputs.logits, dim=1)
    pred_class = torch.argmax(probs).item()
    
    id2label = {
        0: "личная жизнь",
        1: "политика", 
        2: "реклама",
        3: "соцсети",
        4: "спорт",
        5: "юмор"
    }

    for class_id, prob in enumerate(probs.cpu().numpy()[0]):
        print(f"{id2label[class_id]}: {prob:.4f}")  # 4 знака после запятой

In [21]:
class_mapping

{'личная жизнь': 0,
 'политика': 1,
 'реклама': 2,
 'соцсети': 3,
 'спорт': 4,
 'юмор': 5}

In [22]:
predict('Спортивная энергия в каждом глотке — заряжайся и побеждай с новым напитком для чемпионов!')

личная жизнь: 0.0000
политика: 0.0000
реклама: 0.0000
соцсети: 0.0000
спорт: 1.0000
юмор: 0.0000


In [23]:
predict('Достигай рекордов с кроссовками, которые выбирают профи!')

личная жизнь: 0.0000
политика: 0.0000
реклама: 0.0000
соцсети: 0.0000
спорт: 0.0000
юмор: 1.0000


In [1]:
predict('Подпитывай чемпионский дух — белковый батончик для твоих тренировок!')

NameError: name 'predict' is not defined